In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
app_data=pd.read_csv("googleplaystore.csv")
app_review=pd.read_csv("googleplaystore_user_reviews.csv")

In [3]:
app_data.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [4]:
app_data.drop(columns=["Last Updated","Current Ver","Android Ver"],inplace=True)

In [5]:
app_data["Installs"]=app_data["Installs"].str.replace(",","",regex=True)
app_data["Installs"]=app_data["Installs"].str.replace(".$","",regex=True)

In [6]:
app_data=app_data[app_data["Installs"]!="Fre"]
app_data=app_data[app_data["Installs"]!=""]
app_data["Installs"]=pd.to_numeric(app_data["Installs"])

In [7]:
app_data=app_data[app_data["Installs"]>10000]
app_data.drop(columns="Installs",inplace=True)

In [8]:
app_data["Reviews"]=pd.to_numeric(app_data["Reviews"])
app_data=app_data[app_data["Reviews"]>1000]

In [9]:
app_data.drop(columns="Reviews",inplace=True)

In [10]:
app_data["Content Rating"].value_counts()
mask1=app_data["Content Rating"].isin(["Everyone","Teens"])
app_data=app_data[mask1]

In [11]:
app_data.drop(columns=["Size","Type","Price","Content Rating","Genres"],inplace=True)

In [12]:
x=app_data["Category"].value_counts()>50
x=x[x]
listx=x.index.tolist()

In [13]:
app_data=app_data[app_data["Category"].isin(listx)]

In [14]:
app_data.drop(columns="Category",inplace=True)

In [15]:
app_data=app_data[["App","Rating"]]

In [16]:
app_data.dropna(inplace=True)

In [17]:
app_data.shape

(4091, 2)

In [18]:
app_review.shape

(64295, 5)

In [19]:
app_review.dropna(inplace=True)

In [20]:
app_data=app_data.merge(app_review,on="App")

In [21]:
app_review.columns

Index(['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity'],
      dtype='object')

In [22]:
app_data.drop(columns=["Sentiment_Polarity","Sentiment_Subjectivity"],inplace=True)

In [23]:
app_data.shape

(50567, 4)

In [24]:
app_data.drop(columns="App",inplace=True)

In [25]:
y=app_data.iloc[:,0].values

In [26]:
app_data.columns

Index(['Rating', 'Translated_Review', 'Sentiment'], dtype='object')

In [27]:
words=[]
for i in app_data.iloc[:,1]:
    words+=i.split()

In [28]:
len(words)

1100003

In [29]:
preplist=["about","above","across","after","against","among","around","at","before","behind","below","beside","between","by","down","during","except","for","from","in","inside","into","near","of","off","on","out","over","through","to","toward","under","up","with"]
prepcent=[i.capitalize() for i in preplist]
prepupper=[i.upper() for i in preplist]

pronouns=["I","we","you","he","she","it","they","me","us","you","her","him","it","them"]
procent=[i.capitalize() for i in pronouns]
proupper=[i.upper() for i in pronouns]

conjunctions=["for", "and", "nor", "but", "or", "yet", "so", "after", "although", "as", "because", "before", "if", "than", "that", "though", "till", "unless", "until", "when", "whenever", "where", "wherever", "while", "neither", "nor", "either"]
concent=[i.capitalize() for i in conjunctions]
conupper=[i.upper() for i in conjunctions]

determiners=["the","a","an","this", "that", "these", "those", "my", "your", "his", "her", "its", "our", "their", "much", "many", "most", "some", "any", "enough","all", "both", "half", "either", "neither", "each", "every", "other", "another", "such", "what", "rather", "quite"]
detcent=[i.capitalize() for i in determiners]
detupper=[i.upper() for i in determiners]

for i in range(len(words)):
    if not words[i].isalpha():
        words[i]=""
    if ((words[i] in preplist) or (words[i] in prepcent) or(words[i] in prepupper)):
        words[i]=""
    if ((words[i] in pronouns) or (words[i] in procent) or (words[i] in proupper)):
        words[i]=""
    if ((words[i] in conjunctions) or (words[i] in concent) or (words[i] in conupper)):
        words[i]=""
    if ((words[i] in determiners) or (words[i] in detcent) or (words[i] in detupper)):
        words[i]=""


In [30]:
len(words)

1100003

In [31]:
from collections import Counter

In [32]:
word_dict=Counter(words)
del word_dict[""]
len(word_dict)

17659

In [33]:
word_dict=word_dict.most_common(1000)

In [34]:
len(word_dict)

1000

In [35]:
features=[]

In [36]:
for i in app_data.iloc[:,1]:
    blob=i.split()
    data=[]
    
    for j in word_dict:
        data.append(blob.count(j[0]))
    features.append(data)

In [37]:
len(features)

50567

In [38]:
app_data.columns

Index(['Rating', 'Translated_Review', 'Sentiment'], dtype='object')

In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
encoder=LabelEncoder()

In [41]:
app_data["Sentiment"]=encoder.fit_transform(app_data.iloc[:,2].values)

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
oencoder=OneHotEncoder(categorical_features=[0])
labelled=oencoder.fit_transform(app_data.iloc[:,2:].values)

/usr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [44]:
df=pd.DataFrame(labelled.toarray(),dtype="int")

In [45]:
df.drop(columns=0,inplace=True)

In [46]:
nparr=df.values
len(nparr)

50567

In [47]:
X=np.array(features)

In [48]:
#X=np.append(features,df.values,axis=1)

In [49]:
X.shape

(50567, 1000)

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
del nparr #due to low memory

In [52]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

In [53]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regressor=RandomForestRegressor(n_estimators=50)
regressor.fit(X_train,y_train)

In [56]:
y_pred=regressor.predict(X_test)

In [57]:
from sklearn.metrics import r2_score

In [58]:
r2_score(y_test,y_pred)

0.5265493734962852

In [61]:
y_test.shape

(10114,)

In [62]:
y_pred.shape

(10114,)